# Population et conditions sociales

## Population (niveau national) (t_demo_pop)

### Population par pays de naissance - personnes nés à l'étranger (tps00178)

#### Description de l'API
Nombre de personnes nées à l'étranger (selon les frontières actuelles), que ce soit dans d'autres États membres de l'UE ou dans des pays tiers, qui résident habituellement dans le pays déclarant au 1er janvier de l'année concernée.

### Dimensions du Dataset

- **Fréquence Temporelle**
  - Type : Annuel

- **Pays/Région de naissance**
  - Type : Pays étranger

- **Unité de Mesure**
  - Type : Nombre

- **Entité Géopolitique (reporting)**
  - **Entités** :
    - Union européenne - 27 pays (depuis 2020)
    - Zone euro – 20 pays (depuis 2023)
    - Zone euro - 19 pays (2015-2022)
    - Belgique
    - Bulgarie
    - République tchèque
    - Danemark
    - Allemagne
    - Estonie
    - Irlande
    - Grèce
    - Espagne
    - France
    - France métropolitaine
    - Croatie
    - Italie
    - Chypre
    - Lettonie
    - Lituanie
    - Luxembourg
    - Hongrie
    - Malte
    - Pays-Bas
    - Autriche
    - Pologne
    - Portugal
    - Roumanie
    - Slovénie
    - Slovaquie
    - Finlande
    - Suède
    - Islande
    - Liechtenstein
    - Norvège
    - Suisse
    - Royaume-Uni
    - Bosnie-Herzégovine
    - Monténégro
    - Moldavie
    - Macédoine du Nord
    - Géorgie
    - Albanie
    - Serbie
    - Türkiye
    - Ukraine
    - Kosovo*
    - Andorre
    - Biélorussie
    - Monaco
    - Russie
    - Saint-Marin
    - Arménie
    - Azerbaïdjan

- **Temps**
  - **Années** :
    - 2013
    - 2014
    - 2015
    - 2016
    - 2017
    - 2018
    - 2019
    - 2020
    - 2021
    - 2022
    - 2023
    - 2024


In [1]:
#Préparation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score
import statsmodels.api as sm
from scipy import stats
import requests
import eurostat


In [2]:
df = eurostat.get_data_df('tps00178')
df.columns

Index(['freq', 'c_birth', 'age', 'unit', 'sex', 'geo\TIME_PERIOD', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
       '2022', '2023'],
      dtype='object')

In [3]:
df.head()

,freq,c_birth,age,unit,sex,geo\TIME_PERIOD,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,A,FOR,TOTAL,NR,T,AT,1322656.0,1364003.0,1414215.0,1483123.0,1588209.0,1649008.0,1690619.0,1722833.0,1760595.0,1792053.0,1834342.0,1963312.0
1,A,FOR,TOTAL,NR,T,BE,1677737.0,1722265.0,1748748.0,1783488.0,1845631.0,1876726.0,1916272.0,1968060.0,2027763.0,2065727.0,2119691.0,2246910.0
2,A,FOR,TOTAL,NR,T,BG,85076.0,95180.0,107275.0,120635.0,131924.0,139655.0,149096.0,162346.0,176321.0,188519.0,142214.0,168595.0
3,A,FOR,TOTAL,NR,T,CH,2033702.0,2102964.0,2183206.0,2258196.0,2324461.0,2391480.0,2432519.0,2469381.0,2509692.0,2553225.0,2598146.0,2662038.0
4,A,FOR,TOTAL,NR,T,CY,200313.0,200842.0,191587.0,176693.0,172808.0,173791.0,181369.0,185528.0,191415.0,200936.0,205798.0,208843.0


In [7]:
# Dictionnaire des abréviations et noms des pays en français
eurostat_dictionary = {
    
    'BE': 'Belgique',
    'BG': 'Bulgarie',
    'CZ': 'République tchèque',
    'DK': 'Danemark',
    'DE': 'Allemagne',
    'EE': 'Estonie',
    'IE': 'Irlande',
    'EL': 'Grèce',
    'ES': 'Espagne',
    'FR': 'France',
    'HR': 'Croatie',
    'IT': 'Italie',
    'CY': 'Chypre',
    'LV': 'Lettonie',
    'LT': 'Lituanie',
    'LU': 'Luxembourg',
    'HU': 'Hongrie',
    'MT': 'Malte',
    'NL': 'Pays-Bas',
    'AT': 'Autriche',
    'PL': 'Pologne',
    'PT': 'Portugal',
    'RO': 'Roumanie',
    'SI': 'Slovénie',
    'SK': 'Slovaquie',
    'FI': 'Finlande',
    'SE': 'Suède',
    'IS': 'Islande',
    'LI': 'Liechtenstein',
    'NO': 'Norvège',
    'CH': 'Suisse',
    'UK': 'Royaume-Uni',
    'ME': 'Monténégro',
    'MK': 'Macédoine du Nord',
    'AL': 'Albanie',
    'RS': 'Serbie',
    'TR': 'Türkiye',
    
}
# Renommer la colonne 'geo\\time' en 'geo'
df.rename({'geo\\TIME_PERIOD': 'geo'}, inplace=True, axis=1)
# Créer une nouvelle colonne 'pays' avec les noms des pays
df['pays'] = df['geo'].replace(eurostat_dictionary)
print(df.head())

  freq c_birth    age unit sex geo       2012       2013       2014  \
0    A     FOR  TOTAL   NR   T  AT  1322656.0  1364003.0  1414215.0   
1    A     FOR  TOTAL   NR   T  BE  1677737.0  1722265.0  1748748.0   
2    A     FOR  TOTAL   NR   T  BG    85076.0    95180.0   107275.0   
3    A     FOR  TOTAL   NR   T  CH  2033702.0  2102964.0  2183206.0   
4    A     FOR  TOTAL   NR   T  CY   200313.0   200842.0   191587.0   

        2015       2016       2017       2018       2019       2020  \
0  1483123.0  1588209.0  1649008.0  1690619.0  1722833.0  1760595.0   
1  1783488.0  1845631.0  1876726.0  1916272.0  1968060.0  2027763.0   
2   120635.0   131924.0   139655.0   149096.0   162346.0   176321.0   
3  2258196.0  2324461.0  2391480.0  2432519.0  2469381.0  2509692.0   
4   176693.0   172808.0   173791.0   181369.0   185528.0   191415.0   

        2021       2022       2023      pays  
0  1792053.0  1834342.0  1963312.0  Autriche  
1  2065727.0  2119691.0  2246910.0  Belgique  
2   1

In [8]:
# Pivoter le DataFrame jusqu'à 2023
df_melted = df.melt(id_vars=['pays', 'freq', 'c_birth', 'age', 'unit', 'sex', 'geo'], 
                    value_vars=[str(year) for year in range(2012, 2024)],  # Changer 2025 à 2024
                    var_name='année', value_name='valeur')

# Pivoter pour obtenir le format désiré
df_pivoted = df_melted.pivot_table(index='pays', columns='année', values='valeur', aggfunc='first')

# Remplacer NaN par la dernière valeur disponible pour chaque pays (forward fill)
df_pivoted = df_pivoted.ffill(axis=1)

# Réinitialiser l'index pour avoir un DataFrame standard
df_pivoted.reset_index(inplace=True)

# Afficher le résultat
print(df_pivoted)

année                pays       2012       2013       2014        2015  \
0               Allemagne  9117874.0  9456225.0  9807877.0  10220418.0   
1                Autriche  1322656.0  1364003.0  1414215.0   1483123.0   
2                Belgique  1677737.0  1722265.0  1748748.0   1783488.0   
3                Bulgarie    85076.0    95180.0   107275.0    120635.0   
4                  Chypre   200313.0   200842.0   191587.0    176693.0   
5                 Croatie        NaN   569220.0   558092.0    545115.0   
6                Danemark   531540.0   548411.0   569596.0    595876.0   
7               EU27_2020        NaN        NaN        NaN  44279689.0   
8                 Espagne  6294954.0  6165635.0  5952770.0   5883891.0   
9                 Estonie   210842.0   198411.0   196558.0    194664.0   
10               Finlande   260856.0   279743.0   297812.0    314856.0   
11                 France  7463157.0  7543950.0  7746520.0   7915838.0   
12                  Grèce  1312519.0  

In [9]:
# Regrouper par année pour chaque pays
for year in range(2013, 2024):  # De 2013 à 2023
    print(f"Données pour l'année {year}:")
    
    # Sélectionner uniquement les colonnes pertinentes
    df_year = df_pivoted[['pays', str(year)]]
    df_year.rename(columns={str(year): 'valeur'}, inplace=True)

     # Réinitialiser l'index pour éviter les indices numériques
    df_year.reset_index(drop=True, inplace=True)
    
    # Afficher les données
    print(df_year.head())  # Utiliser head() pour afficher les premières lignes
    print("\n")  # Ajoute un saut de ligne pour la lisibilité

Données pour l'année 2013:
année       pays     valeur
0      Allemagne  9456225.0
1       Autriche  1364003.0
2       Belgique  1722265.0
3       Bulgarie    95180.0
4         Chypre   200842.0


Données pour l'année 2014:
année       pays     valeur
0      Allemagne  9807877.0
1       Autriche  1414215.0
2       Belgique  1748748.0
3       Bulgarie   107275.0
4         Chypre   191587.0


Données pour l'année 2015:
année       pays      valeur
0      Allemagne  10220418.0
1       Autriche   1483123.0
2       Belgique   1783488.0
3       Bulgarie    120635.0
4         Chypre    176693.0


Données pour l'année 2016:
année       pays      valeur
0      Allemagne  10908255.0
1       Autriche   1588209.0
2       Belgique   1845631.0
3       Bulgarie    131924.0
4         Chypre    172808.0


Données pour l'année 2017:
année       pays      valeur
0      Allemagne  12105436.0
1       Autriche   1649008.0
2       Belgique   1876726.0
3       Bulgarie    139655.0
4         Chypre    173791.0

C:\Users\hp\AppData\Local\Temp\ipykernel_11804\2128398066.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year.rename(columns={str(year): 'valeur'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_11804\2128398066.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year.rename(columns={str(year): 'valeur'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_11804\2128398066.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [10]:
for year in range(2013, 2024):  # De 2013 à 2023
    # Récupérer les données pour l'année spécifique
    df_year = df_pivoted[['pays', str(year)]].copy()  # Créer une copie explicite
    df_year.rename(columns={str(year): 'valeur'}, inplace=True)
    
    # Enregistrer le DataFrame dans un fichier CSV
    file_name = f"population_par_pays_de_naissance_donnees_{year}.csv"  # Nom du fichier pour l'année
    df_year.to_csv(file_name, index=False, encoding='utf-8')
    print(f"Fichier CSV généré : {file_name}")

Fichier CSV généré : population_par_pays_de_naissance_donnees_2013.csv
Fichier CSV généré : population_par_pays_de_naissance_donnees_2014.csv
Fichier CSV généré : population_par_pays_de_naissance_donnees_2015.csv
Fichier CSV généré : population_par_pays_de_naissance_donnees_2016.csv
Fichier CSV généré : population_par_pays_de_naissance_donnees_2017.csv
Fichier CSV généré : population_par_pays_de_naissance_donnees_2018.csv
Fichier CSV généré : population_par_pays_de_naissance_donnees_2019.csv
Fichier CSV généré : population_par_pays_de_naissance_donnees_2020.csv
Fichier CSV généré : population_par_pays_de_naissance_donnees_2021.csv
Fichier CSV généré : population_par_pays_de_naissance_donnees_2022.csv
Fichier CSV généré : population_par_pays_de_naissance_donnees_2023.csv
